In [ ]:
import warnings
import numpy as np
from tqdm.notebook import tqdm
from tabulate import tabulate
from lightsim2grid.solver import SolverType
from lightsim2grid.securityAnalysis import SecurityAnalysisCPP
from lightsim2grid.gridmodel import init
import time
import pandapower as pp
import pandapower.networks as pn
import plotly.graph_objects as go
# ordered per number of branches
case_names = ["case14", "case118", "case_illinois200", "case300", "case1354pegase",
              "case1888rte", "GBnetwork", "case3120sp", "case2848rte", "case2869pegase", 
            #   "case6495rte", "case6515rte", "case9241pegase"
             ]

In [ ]:
def compute_lightsim2grid(case):
    """compute the full security analysis using lightsim2grid"""
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        gridmodel = init(case)
    # perform the action
    # .XXX(blablabla)
    
    # start the initial computation
    V = gridmodel.dc_pf(1.04 * np.ones(case.bus.shape[0], dtype=np.complex128), 10, 1e-7)
    if not V.shape:
        # ac pf has diverged
        warnings.warn(f"Impossible to compute the security analysis for {case_nm}: divergence")
    
    # initial the model
    sec_analysis = SecurityAnalysisCPP(gridmodel)
    sec_analysis.change_solver(SolverType.KLUDC)
    for branch_id in range(len(gridmodel.get_lines()) + len(gridmodel.get_trafos())):
        sec_analysis.add_n1(branch_id)
    
    # now do the security analysis
    beg = time.perf_counter()
    sec_analysis.compute(V, 10, 1e-7)
    vs_sa = sec_analysis.get_voltages()
    mw_sa = sec_analysis.compute_power_flows()
    tot_time = time.perf_counter() - beg
    return tot_time, sec_analysis.nb_solved()
    

In [ ]:
def compute_pandapower(case):
    pp.rundcpp(case)  # run initial powerflow
    
    nb_cont = 0
    tot_time = 0.
    # now do the security analysis
    for branch_id in range(case.line.shape[0]):
        beg = time.perf_counter()
        case.line["in_service"].iloc[branch_id] = False
        pp.rundcpp(case)
        if case["_ppc"]["success"]:
            tot_time += time.perf_counter() - beg
            nb_cont += 1
        case.line["in_service"].iloc[branch_id] = True
        
    for branch_id in range(case.trafo.shape[0]):
        beg = time.perf_counter()
        case.trafo["in_service"].iloc[branch_id]= False
        pp.rundcpp(case)
        if case["_ppc"]["success"]:
            tot_time += time.perf_counter() - beg
            nb_cont += 1
        case.trafo["in_service"].iloc[branch_id] = True
    return tot_time, nb_cont

In [ ]:
res_table = []
res_per_cont = []
nb_branch = []
for case_nm in tqdm(case_names):
    this_row = [case_nm, None, None, None]  # total time
    this_row_per_cont = [case_nm, None, None, None]  # time for a single contingency
    
    # retrieve the case file from pandapower
    case = getattr(pn, case_nm)()
    nb_branch.append(case.line.shape[0] + case.trafo.shape[0])
    
    # use lightsim2grid
    total_time, nb_cont = compute_lightsim2grid(case)
    this_row[2] = total_time
    this_row_per_cont[2] = total_time / nb_cont
    ##### end lightsim2grid
    
    # use pandapower
    total_time, nb_cont = compute_pandapower(case)
    this_row[3] = total_time
    this_row_per_cont[3] = total_time / nb_cont
    ##### end pandapower
    
    res_table.append(this_row)
    res_per_cont.append(this_row_per_cont)

In [ ]:
print(tabulate(res_table, headers=["Grid", "Ext ST", "lightsim2grid", "pandapower"], floatfmt=".2e"))

In [ ]:
print(tabulate(res_per_cont, headers=["Grid", "Ext ST", "lightsim2grid", "pandapower"], floatfmt=".2e"))

In [ ]:
print(tabulate(res_table, headers=["Grid", "Ext ST", "lightsim2grid", "pandapower"], tablefmt="latex", floatfmt=".2e"))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Line(x=nb_branch, y=[el[2] for el in res_table], name="lightsim2grid"))
fig.add_trace(go.Line(x=nb_branch, y=[el[3] for el in res_table], name="pandapower"))
# fig.add_trace(go.Line(x=nb_branch, y=nb_branch))
fig.update_xaxes(title_text="Grid size (number of branch)")
fig.update_yaxes(title_text="Computation time (s)")
fig.update_yaxes(type="log")
fig

In [ ]:
fig = go.Figure()
fig.add_trace(go.Line(x=nb_branch, y=[el[2] for el in res_per_cont], name="lightsim2grid"))
fig.add_trace(go.Line(x=nb_branch, y=[el[3] for el in res_per_cont], name="pandapower"))
# fig.add_trace(go.Line(x=nb_branch, y=nb_branch, name="test"))
fig.update_xaxes(title_text="Grid size (number of branch)")
fig.update_yaxes(title_text="Computation time per cont. (s) [log scale]")
fig.update_yaxes(type="log")
fig